<a href="https://colab.research.google.com/github/KrisrcQi/Crsp_mutual_fund_matrix_completion/blob/main/Test_of_matrix_NAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import pandas as pd
import numpy as np


# Reading the data file
# Checking the empty cells in the dataset
df = pd.read_csv('Downsized dataset(10k row).csv')
print(df)
small_date = df['caldt'].min()
print(small_date)
max_date = df['caldt'].max()
print(max_date)



      Unnamed: 0       caldt  crsp_fundno  mtna     mnav
0              0  28/06/2019        95598   0.1  10.0100
1              1  31/07/2019        95598   0.1  10.0100
2              2  30/08/2019        95598   0.1  10.0200
3              3  30/09/2019        95598   0.1  10.0200
4              4  31/10/2019        95598   0.1  10.0200
...          ...         ...          ...   ...      ...
9980        9980  30/04/2020        96709   2.3  10.8100
9981        9981  31/03/2020        96710   0.1  10.0000
9982        9982  30/04/2020        96710   0.1  10.8100
9983        9983  31/03/2020        96711   4.0  40.0000
9984        9984  30/04/2020        96711  13.8  44.3728

[9985 rows x 5 columns]
26/02/2010
31/12/2019


In [90]:
df_1 = df[: 633]
df_1 = df.drop(['caldt'], axis=1)
df_1

,Unnamed: 0,crsp_fundno,mtna,mnav
0,0,95598,0.1,10.0100
1,1,95598,0.1,10.0100
2,2,95598,0.1,10.0200
3,3,95598,0.1,10.0200
4,4,95598,0.1,10.0200
...,...,...,...,...
9980,9980,96709,2.3,10.8100
9981,9981,96710,0.1,10.0000
9982,9982,96710,0.1,10.8100
9983,9983,96711,4.0,40.0000


In [ ]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 11.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=3445bb49895e4a89aff3a93b6774b194080a8de877b4bf92c35b0aaf345ff0f9
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric


In [91]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv

data = df_1
num_features = 10

# Step 1: Convert matrix to graph representation
adjacency_matrix = torch.tensor(data.values)  # Convert matrix to tensor
edge_index = adjacency_matrix.nonzero().t()  # Get indices of non-zero values as edges
x = torch.randn(adjacency_matrix.size(0), num_features)  # Node features (randomly initialized)


In [92]:
adjacency_matrix


tensor([[0.00e+00, 9.56e+04, 1.00e-01, 1.00e+01],
        [1.00e+00, 9.56e+04, 1.00e-01, 1.00e+01],
        [2.00e+00, 9.56e+04, 1.00e-01, 1.00e+01],
        ...,
        [9.98e+03, 9.67e+04, 1.00e-01, 1.08e+01],
        [9.98e+03, 9.67e+04, 4.00e+00, 4.00e+01],
        [9.98e+03, 9.67e+04, 1.38e+01, 4.44e+01]], dtype=torch.float64)

In [ ]:
edge_index

tensor([[   0,    0,    0,  ..., 9984, 9984, 9984],
        [   1,    2,    3,  ...,    1,    2,    3]])

In [ ]:
x

tensor([[-0.75, -0.21, -0.05,  ..., -2.18,  0.40, -0.68],
        [-1.45,  0.91, -0.21,  ...,  0.46,  0.27,  0.35],
        [-1.86, -1.65,  1.20,  ...,  0.15, -0.57, -1.19],
        ...,
        [ 1.43,  1.13, -0.54,  ...,  0.64, -0.90, -0.68],
        [-2.75,  0.51, -0.04,  ...,  0.34,  1.37, -2.48],
        [ 1.37,  0.54, -0.23,  ...,  0.40,  0.49,  1.71]])

In [93]:
# Step 2: Split the dataset
# Randomly remove some values to create observed values dataset
observed_matrix = adjacency_matrix.clone()
indices = torch.randperm(adjacency_matrix.numel())[:int(0.8 * adjacency_matrix.numel())]
observed_matrix.reshape(-1)[indices] = float('nan')
# Keep missing values as the evaluation dataset
missing_values = adjacency_matrix.clone()
missing_values.reshape(-1)[indices] = float('nan')

In [94]:
# Step 3: Construct the GNN model
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.fc(x).squeeze()
        return x

input_dim = num_features  # Dimensionality of input features
hidden_dim = 64  # Dimensionality of hidden layers

model = GNNModel(input_dim, hidden_dim)

In [97]:
# Step 4: Train the GNN model
criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model.parameters(), lr=0.01)

data = Data(x=x, edge_index=edge_index)
loader = DataLoader([data], batch_size=64)  # DataLoader for a single graph

num_epochs = 40

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in loader:
        optimizer.zero_grad()
        output = model(batch.x, batch.edge_index)
        target = batch.x.t()  # Transpose the target tensor to match output shape
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss}")

Epoch [1/40], Loss: 1.6566777229309082
Epoch [2/40], Loss: 0.9538108706474304
Epoch [3/40], Loss: 1.102125883102417
Epoch [4/40], Loss: 1.0525873899459839
Epoch [5/40], Loss: 0.9830942749977112
Epoch [6/40], Loss: 0.93965744972229
Epoch [7/40], Loss: 0.9285320043563843
Epoch [8/40], Loss: 0.9355182647705078
Epoch [9/40], Loss: 0.9471778273582458
Epoch [10/40], Loss: 0.9556674361228943
Epoch [11/40], Loss: 0.9583871364593506
Epoch [12/40], Loss: 0.9555902481079102
Epoch [13/40], Loss: 0.9490392804145813
Epoch [14/40], Loss: 0.9407772421836853
Epoch [15/40], Loss: 0.9324722290039062
Epoch [16/40], Loss: 0.9252710342407227
Epoch [17/40], Loss: 0.9198061227798462
Epoch [18/40], Loss: 0.9158070683479309
Epoch [19/40], Loss: 0.9125461578369141
Epoch [20/40], Loss: 0.9092729687690735
Epoch [21/40], Loss: 0.905791163444519
Epoch [22/40], Loss: 0.9024783968925476
Epoch [23/40], Loss: 0.9002458453178406
Epoch [24/40], Loss: 0.899595320224762
Epoch [25/40], Loss: 0.8997943997383118
Epoch [26/40],

In [98]:
# Step 5: Evaluate the GNN model
model.eval()
with torch.no_grad():
    missing_values_prediction = model(x, edge_index)  # Predict missing values


# Convert the predicted missing values back to a matrix
predicted_matrix = adjacency_matrix.clone().double()  # Convert predicted_matrix to Double data type
selected_indices = indices[:missing_values_prediction.size(0)]
predicted_matrix.reshape(-1)[selected_indices] = missing_values_prediction.double()  # Convert missing_values_prediction to Double data type



In [99]:
# Print the predicted matrix
torch.set_printoptions(precision=2)
print(predicted_matrix)

tensor([[0.00e+00, 9.56e+04, 1.00e-01, 1.00e+01],
        [1.00e+00, 9.56e+04, 1.00e-01, 1.00e+01],
        [2.00e+00, 9.56e+04, 1.00e-01, 1.00e+01],
        ...,
        [9.98e+03, 9.67e+04, 1.00e-01, 1.08e+01],
        [9.98e+03, 9.67e+04, 4.00e+00, 4.00e+01],
        [9.98e+03, 9.67e+04, 1.38e+01, 4.44e+01]], dtype=torch.float64)


In [100]:
# Convert the tensor to a NumPy array
predicted_array = predicted_matrix.numpy()

# Create a new DataFrame from the array
predicted_df = pd.DataFrame(predicted_array)

# Print the new DataFrame
print(predicted_df)

           0        1     2        3
0        0.0  95598.0   0.1  10.0100
1        1.0  95598.0   0.1  10.0100
2        2.0  95598.0   0.1  10.0200
3        3.0  95598.0   0.1  10.0200
4        4.0  95598.0   0.1  10.0200
...      ...      ...   ...      ...
9980  9980.0  96709.0   2.3  10.8100
9981  9981.0  96710.0   0.1  10.0000
9982  9982.0  96710.0   0.1  10.8100
9983  9983.0  96711.0   4.0  40.0000
9984  9984.0  96711.0  13.8  44.3728

[9985 rows x 4 columns]


In [ ]:
predicted_df.to_csv('predicted matrix.csv')